## <font color='red'>Depecrated for now, may cause issues</font>

# Step 1

In [6]:
from __future__ import absolute_import, division, print_function
from NuRadioReco.utilities import units
from NuRadioMC.EvtGen.generator import generate_eventlist_cylinder

# Setup logging
from NuRadioReco.utilities.logging import setup_logger
logger = setup_logger(name="")

# define simulation volume (artificially close by to make them trigger)
volume = {
'fiducial_zmin':-1 * units.km,  # the ice sheet at South Pole is 2.7km deep
'fiducial_zmax': 0 * units.km,
'fiducial_rmin': 0 * units.km,
'fiducial_rmax': 1 * units.km}

# generate one event list at 1e19 eV with 1000 neutrinos
generate_eventlist_cylinder('1e19_n1e3.hdf5', 1e2, 1e19 * units.eV, 1e19 * units.eV, volume)

fatal: your current branch 'master' does not have any commits yet
INFO - NuRadioMC.EvtGen - increasing rmax from 1.0km to 1.0km, zmax from 0.0km to 0.0km
INFO - NuRadioMC.EvtGen - increasing number of events from 100 to 100
INFO - NuRadioMC.EvtGen - processing batch 1/1 with 1e+02 events
INFO - NuRadioMC.EvtGen - Time per event (PROPOSAL only): 0.0000 ms
INFO - NuRadioMC.EvtGen - Total time (PROPOSAL only) 0s
INFO - NuRadioMC.EvtGen - number of fiducial showers 122
INFO - NuRadioMC.EvtGen - saving 100 events in total
STATUS - NuRadioMC.EvtGen - writing file 1e19_n1e3.hdf5 with 100 events (id 1 - 100) and 122 entries
INFO - NuRadioMC.EvtGen - wrote 100 events in total
STATUS - NuRadioMC.EvtGen - finished in 0s


# Step 2

In [7]:
from __future__ import absolute_import, division, print_function
import argparse
# import detector simulation modules
import NuRadioReco.modules.trigger.highLowThreshold
import NuRadioReco.modules.trigger.simpleThreshold
import NuRadioReco.modules.channelBandPassFilter
from NuRadioReco.utilities import units
from NuRadioMC.simulation import simulation

# Setup logging
from NuRadioReco.utilities.logging import setup_logger
logger = setup_logger(name="")

# initialize detector sim modules
simpleThreshold = NuRadioReco.modules.trigger.simpleThreshold.triggerSimulator()
highLowThreshold = NuRadioReco.modules.trigger.highLowThreshold.triggerSimulator()
channelBandPassFilter = NuRadioReco.modules.channelBandPassFilter.channelBandPassFilter()

# Set to true for default simulation. IF FALSE WILL NOT RUN IN ipynb!
default = True

class mySimulation(simulation.simulation):

    def _detector_simulation_filter_amp(self, evt, station, det):
        channelBandPassFilter.run(evt, station, det, passband=[80 * units.MHz, 1000 * units.GHz],
                                  filter_type='butter', order=2)
        channelBandPassFilter.run(evt, station, det, passband=[0, 500 * units.MHz],
                                  filter_type='butter', order=10)

    def _detector_simulation_trigger(self, evt, station, det):
        # first run a simple threshold trigger
        simpleThreshold.run(evt, station, det,
                             threshold=3 * self._Vrms,
                             triggered_channels=None,  # run trigger on all channels
                             number_concidences=1,
                             trigger_name='simple_threshold')  # the name of the trigger

        # run a high/low trigger on the 4 downward pointing LPDAs
        highLowThreshold.run(evt, station, det,
                                    threshold_high=4 * self._Vrms,
                                    threshold_low=-4 * self._Vrms,
                                    triggered_channels=[0, 1, 2, 3],  # select the LPDA channels
                                    number_concidences=2,  # 2/4 majority logic
                                    trigger_name='LPDA_2of4_4.1sigma',
                                    set_not_triggered=(not station.has_triggered("simple_threshold")))  # calculate more time consuming ARIANNA trigger only if station passes simple trigger

if default is True:
    sim = mySimulation(inputfilename='1e19_n1e3.hdf5',
                            outputfilename='station.json',
                            detectorfile='config.yaml',
                            outputfilenameNuRadioReco='output.hdf5',
                            config_file='output.nur',
                            file_overwrite=True)
    sim.run()
else:
    parser = argparse.ArgumentParser(description='Run NuRadioMC simulation')
    parser.add_argument('inputfilename', type=str,
                        help='path to NuRadioMC input event list')
    parser.add_argument('detectordescription', type=str,
                        help='path to file containing the detector description')
    parser.add_argument('config', type=str,
                        help='NuRadioMC yaml config file')
    parser.add_argument('outputfilename', type=str,
                        help='hdf5 output filename')
    parser.add_argument('outputfilenameNuRadioReco', type=str, nargs='?', default=None,
                        help='outputfilename of NuRadioReco detector sim file')
    args = parser.parse_args()

    if __name__ == "__main__":
        sim = mySimulation(inputfilename=args.inputfilename,
                                    outputfilename=args.outputfilename,
                                    detectorfile=args.detectordescription,
                                    outputfilenameNuRadioReco=args.outputfilenameNuRadioReco,
                                    config_file=args.config,
                                    file_overwrite=True)
        sim.run() 

STATUS - NuRadioMC.simulation - reading default config from /data/i3home/ssued/lib/python3.12/site-packages/NuRadioMC/simulation/config_default.yaml
STATUS - NuRadioMC.simulation - reading local config overrides from output.nur


FileNotFoundError: [Errno 2] No such file or directory: 'output.nur'